In [6]:

#r "nuget:HtmlAgilityPack, 1.11.65"
#r "nuget:ReverseMarkdown, 4.6.0"
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.FileExtensions, 8.0.1"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.33.0"
#r "nuget: Microsoft.SemanticKernel.Connectors.Google, 1.33.0-alpha"

using HtmlAgilityPack;
using ReverseMarkdown;
using Microsoft.Extensions.Configuration;
using System.IO;


using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

public static var config = new ConfigurationBuilder()
          .AddJsonFile(Path.GetFullPath("secrets.json"), optional: false, reloadOnChange: true)
          .Build();

public static string AIKey =  config["gemini-key"];

Installed Packages HtmlAgilityPack, 1.11.65 Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.FileExtensions, 8.0.1 Microsoft.Extensions.Configuration.Json, 8.0.0 Microsoft.SemanticKernel, 1.33.0 Microsoft.SemanticKernel.Connectors.Google, 1.33.0-alpha ReverseMarkdown, 4.6.0

In [ ]:
var url = "https://bradjolicoeur.com/article/html-to-json-semantic-kernel";
var uri = new Uri(url);

// Get the URL specified
var webGet = new HtmlWeb();
var document = await webGet.LoadFromWebAsync(url);
var body =  document.DocumentNode.SelectSingleNode("/html/body");
var metaTags = document.DocumentNode.SelectNodes("//meta");
var metaText = string.Empty;


In [8]:
if (metaTags != null)
{
    var sb = new StringBuilder();
    foreach (var item in metaTags)
    {
        sb.Append(item.GetAttributeValue("property", ""));
        sb.Append('|');
        sb.Append(item.GetAttributeValue("content", ""));
        sb.AppendLine();
    }
    metaText = sb.ToString();
}


In [9]:
var config = new ReverseMarkdown.Config{
    UnknownTags = Config.UnknownTagsOption.Drop
};

var converter = new ReverseMarkdown.Converter(config);
string html = body.OuterHtml;

string markdownText = converter.Convert(html);


In [12]:
var sbAllText = new StringBuilder();
sbAllText.AppendLine(metaText);
sbAllText.AppendLine(markdownText);
var textToSummarize = sbAllText.ToString();

textToSummarize

|IE=edge
|ApvK67ociHgr2egd6c2ZjrfPuRs8BHcvSggogIOPQNH7GJ3cVlyJ1NOq/COCdj0+zxskqHt9HgLLETc8qqD+vwsAAABteyJvcmlnaW4iOiJodHRwczovL3lvdXR1YmUuY29tOjQ0MyIsImZlYXR1cmUiOiJQcml2YWN5U2FuZGJveEFkc0FQSXMiLCJleHBpcnkiOjE2OTUxNjc5OTksImlzU3ViZG9tYWluIjp0cnVlfQ==
|rgba(255, 255, 255, 0.98)
|Bolt.DIY + Gemini 2.0 Pro: Develop a Full-stack App 100% FREE Without Writing Code! (OPENSOURCE)
|Want to build full-stack web apps without writing a single line of code? With Bolt.DIY and Gemini 2.0 Pro, you can prompt, run, edit, and deploy applications...
|bolt diy, cursor alternative, ai coding, bolt.diy, gemini 2.0 pro, no-code development, full-stack ai, open-source ai, build apps with ai, llm coding, Bolt.DIY + Deepseek-R1, bolt.new + deepseek r1, deepseek r1, bolt free alternative, gemini new ai coder, bolt diy new ai coder, fully free new ai coder, opensource alternative to v0, opensource alternative alternative to bolt, text to application, text to frontend, claude alternative, cline, free api, free ge

In [11]:
#pragma warning disable SKEXP0070
#pragma warning disable SKEXP0010

static string ModelId = "gemini-1.5-pro";

// Create a kernel with OpenAI chat completion
var kernel = Kernel.CreateBuilder().AddGoogleAIGeminiChatCompletion(ModelId,AIKey).Build();
//var builder = Kernel.CreateBuilder()
//.AddOpenAIChatCompletion(ModelId, OpenAIKey);

//Kernel kernel = builder.Build();

// Create and print out the prompt
string prompt = $"""     
    Please summarize the the following text in 50 words or less for an summary to use in LinkedIn Featured section:
    {textToSummarize}

    Output the summary along with the other properties in the JSON schema below.

    Use this JSON Schema:
    Article = 'Author':str, 'PublishDate':datetime, 'Title':str, 'Summary':str, 'KeyWords':str, 'ImageUrl':str 

    """;

// Submit the prompt and print out the response
string response = await kernel.InvokePromptAsync<string>(
    prompt,
    new(new OpenAIPromptExecutionSettings() 
        { 
            MaxTokens = 1000,
            ResponseFormat = "json_object"
        })
    );

response

```json
{
  "Author": "@youtube",
  "PublishDate": "2025-02-07T17:14:59-08:00",
  "Title": "Bolt.DIY + Gemini 2.0 Pro: Develop a Full-stack App 100% FREE Without Writing Code! (OPENSOURCE)",
  "Summary": "Build full-stack web apps without coding using Bolt.DIY and Gemini 2.0 Pro.  Prompt, run, edit, and deploy applications 100% free and open-source.",
  "KeyWords": "bolt diy, gemini 2.0 pro, no-code development, full-stack ai, open-source ai, ai coding, cursor alternative",
  "ImageUrl": "https://i.ytimg.com/vi/nYg-sZB6jxI/maxresdefault.jpg"
}
```
